In [1]:
import os
import importlib
from utility import Pi_util
from natsort import natsorted
import pickle
import sys
import pandas as pd

path_to_lib: str = "/Users/zacharykelly/Documents/MATLAB/projects/combiExperiments/code/lightLogger/miniSpect"
sys.path.append(path_to_lib)
import MS_util
import importlib

In [2]:
experiment_path: str = "/Volumes/EXTERNAL1/queue_5hz_0NDF"

In [4]:
importlib.reload(Pi_util)
parsed = Pi_util.parse_chunks_pkl(experiment_path, use_mean_frame=True)

print(parsed[0].keys())

Loading chunk: 1/30
Loading chunk: 2/30
Loading chunk: 3/30
Loading chunk: 4/30
Loading chunk: 5/30
Loading chunk: 6/30
Loading chunk: 7/30
Loading chunk: 8/30
Loading chunk: 9/30
Loading chunk: 10/30
Loading chunk: 11/30
Loading chunk: 12/30
Loading chunk: 13/30
Loading chunk: 14/30
Loading chunk: 15/30
Loading chunk: 16/30
Loading chunk: 17/30
Loading chunk: 18/30
Loading chunk: 19/30
Loading chunk: 20/30
Loading chunk: 21/30
Loading chunk: 22/30
Loading chunk: 23/30
Loading chunk: 24/30
Loading chunk: 25/30
Loading chunk: 26/30
Loading chunk: 27/30
Loading chunk: 28/30
Loading chunk: 29/30
Loading chunk: 30/30
Parsing chunk: 1/30
Length of world vals: 4
Frame Buffer Shape: (2000,)
Captured Frames: 2006 | FPS: 200.57450679074046
Length of MS Vals: 3
Reading Buffer Shape: (10, 150)
Captured Frames: 12 | FPS: 1.1496680085416646
Length of Pupil vals: 3
Frame Buffer Shape: (1200,)
Captured Frames: 1189 | FPS: 118.81791563216787
Parsing chunk: 2/30
Length of world vals: 4
Frame Buffer Sha

In [3]:
"""Parser for the raw World data per chunk"""
def world_parser(val_tuple: tuple):
    print(f'Length of world vals: {len(val_tuple)}')

    # First value is always the frame buffer for this chunk 
    frame_buffer: np.ndarray = val_tuple[0]

    # Second value is always the settings buffer for this chunk
    # The settings are in the format [duration, FPS gain, exposure] TODO: The FPS dimension does not currently exist, but going to add it
    settings_buffer: np.ndarray = val_tuple[1]

    # Third and Fourth values are always the num_captured_frames and observed FPS 
    num_captured_frames, observed_fps = val_tuple[2:]

    print(f'Frame Buffer Shape: {frame_buffer.shape}')
    print(f'Captured Frames: {num_captured_frames} | FPS: {observed_fps}')
                                                # Make this a float for MATLAB use later
    return {'frame_buffer': frame_buffer, 'settings_buffer': settings_buffer, 'num_frames_captured': float(num_captured_frames), 'FPS': observed_fps}

"""Parser for the raw Pupil data per chunk"""
def pupil_parser(val_tuple: tuple):
    print(f'Length of Pupil vals: {len(val_tuple)}')

    # First value is always the frame buffer for this chunk
    frame_buffer: np.ndarray = val_tuple[0]

    # Second value and third value are always num_captured_frames and observed FPS
    num_captured_frames, observed_fps = val_tuple[1:]

    print(f'Frame Buffer Shape: {frame_buffer.shape}')
    print(f'Captured Frames: {num_captured_frames} | FPS: {observed_fps}')
                                            # Make this a float for MATLAB use later
    return {'frame_buffer': frame_buffer, 'num_frames_captured': float(num_captured_frames), 'FPS': observed_fps}



"""Parser for the raw MS data per chunk"""
def ms_parser(val_tuple: tuple):    
    print(f'Length of MS Vals: {len(val_tuple)}')
    
    # First value is always the unparsed byte buffer 
    bytes_buffer: np.ndarray = val_tuple[0]

    # Use the MS util parsing library to unpack these bytes
    #AS_channels, TS_channels, LS_channels, LS_temp = MS_util.parse_SERIAL(bytes_buffer)

    # TODO: Still need to parse this fully (like, unpack the buffer of accelerometer values, for instance, and make into DFs)

    # Second and third values are always the num_captured_frames and observed FPS 
    num_captured_frames, observed_fps = val_tuple[1:]
    print(f'Reading Buffer Shape: {bytes_buffer.shape}')
    print(f'Captured Frames: {num_captured_frames} | FPS: {observed_fps}')

    return {'readings_buffer': float(0)}


In [4]:
# Initialize a list to hold the sorted chunks after 
# they have been parsed 
sorted_chunks: list = []

# Define a dictionary of sensor initials and their respective parsers 
sensor_parsers: dict = {'W': world_parser, 
                       'P': pupil_parser,
                       'M': ms_parser}

# First, we must find the gather the sorted paths to the chunks
# which are stored in .pkl files
chunk_paths: list = natsorted([os.path.join(experiment_path, file) 
                                for file in os.listdir(experiment_path)
                                if '.pkl' in file])

print(chunk_paths)

['/Volumes/EXTERNAL1/queue_5hz_0NDF/0.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/1.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/2.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/3.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/4.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/5.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/6.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/7.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/8.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/9.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/10.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/11.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/12.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/13.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/14.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/15.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/16.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/17.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/18.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/19.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/20.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/21.pkl', '/Volumes/EXTERNAL1/queue_5hz_0NDF/22.pkl

In [5]:
# Next, we will iterate over the chunk files and load them in 
for chunk_num, path in enumerate(chunk_paths):
    print(f'Loading chunk: {chunk_num+1}/{len(chunk_paths)}')

    # Read in the file and append it to the sorted chunks 
    # list
    with open(path, 'rb') as f:
        # Read in the dictionary of values from this chunk
        chunk_dict: dict = pickle.load(f)

        # Append it to the sorted chunk list 
        sorted_chunks.append(chunk_dict)

print(sorted_chunks)

Loading chunk: 1/30
Loading chunk: 2/30
Loading chunk: 3/30


/var/folders/fb/55l_tk897yjbwpmc6_mm1ylm0000gp/T/ipykernel_1635/3123933375.py:9: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  chunk_dict: dict = pickle.load(f)


Loading chunk: 4/30
Loading chunk: 5/30
Loading chunk: 6/30
Loading chunk: 7/30
Loading chunk: 8/30
Loading chunk: 9/30
Loading chunk: 10/30
Loading chunk: 11/30
Loading chunk: 12/30
Loading chunk: 13/30
Loading chunk: 14/30
Loading chunk: 15/30
Loading chunk: 16/30
Loading chunk: 17/30
Loading chunk: 18/30
Loading chunk: 19/30
Loading chunk: 20/30
Loading chunk: 21/30
Loading chunk: 22/30
Loading chunk: 23/30
Loading chunk: 24/30
Loading chunk: 25/30
Loading chunk: 26/30
Loading chunk: 27/30
Loading chunk: 28/30
Loading chunk: 29/30
Loading chunk: 30/30
[{'W': (array([[[[187,  63, 195, ...,  60, 185,  59],
         [ 58,  57,  58, ...,  56,  57,  55],
         [186,  63, 192, ...,  57, 185,  58],
         ...,
         [ 51,  53,  51, ...,  54,  51,  53],
         [171,  58, 176, ...,  54, 178,  56],
         [ 52,  52,  52, ...,  53,  52,  53]],

        [[179,  63, 186, ...,  57, 180,  58],
         [ 57,  55,  57, ...,  55,  55,  54],
         [181,  61, 185, ...,  55, 181,  56],
 

In [7]:
# Next, we will iterate over the chunks and their sensors and their respective data in the chunk and parse them 
parsed_chunks: list = []
for chunk_num, chunk in enumerate(sorted_chunks):
    print(f'Parsing chunk: {chunk_num+1}/{len(sorted_chunks)}')
    # initialize a new dictionary to hold sensors' parsed information
    parsed_chunk: dict = {}

    for key, val in chunk.items():
        # Parse this sensor's data with its appropriate sensor
        parsed_data: dict = sensor_parsers[key](val)

        # Note this sensor's parsed info for this chunk 
        parsed_chunk[key] = parsed_data
    
    # Append this parsed chunk to the growing list of parsed chunks 
    parsed_chunks.append(parsed_chunk)


print(parsed_chunks)



Parsing chunk: 1/30
Length of world vals: 4
Frame Buffer Shape: (10, 200, 30, 40)
Captured Frames: 2006 | FPS: 200.57450679074046
Length of MS Vals: 3
Reading Buffer Shape: (10, 150)
Captured Frames: 12 | FPS: 1.1496680085416646
Length of Pupil vals: 3
Frame Buffer Shape: (10, 120, 400, 400)
Captured Frames: 1189 | FPS: 118.81791563216787
Parsing chunk: 2/30
Length of world vals: 4
Frame Buffer Shape: (10, 200, 30, 40)
Captured Frames: 2006 | FPS: 200.55300686475118
Length of MS Vals: 3
Reading Buffer Shape: (10, 150)
Captured Frames: 11 | FPS: 1.0533067837866343
Length of Pupil vals: 3
Frame Buffer Shape: (10, 120, 400, 400)
Captured Frames: 1202 | FPS: 120.12241009868637
Parsing chunk: 3/30
Length of world vals: 4
Frame Buffer Shape: (10, 200, 30, 40)
Captured Frames: 2006 | FPS: 200.4328079545044
Length of MS Vals: 3
Reading Buffer Shape: (10, 150)
Captured Frames: 11 | FPS: 1.056859908155557
Length of Pupil vals: 3
Frame Buffer Shape: (10, 120, 400, 400)
Captured Frames: 1201 | FPS